In [1]:
import networkx as nx
from pyvis.network import Network
import random
import os


In [14]:

def load_random_graph(directory):
    files = [file for file in os.listdir(directory) if file.endswith('.graphml')] # or other format
    random_file = random.choice(files)
    path = os.path.join(directory, random_file)
    return nx.read_graphml(path)  # Change this according to your graph format

def get_root_nodes(G):
    return [n for n, d in G.in_degree() if d == 0]


def get_target_nodes(root, G):
    target_nodes = [node for node, attributes in G.nodes(data=True) if attributes['cat'] == '1']
    return target_nodes
    



In [31]:
directory = '/home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24'
G = load_random_graph(directory)
root_nodes = get_root_nodes(G)
target_nodes = get_target_nodes(root_nodes, G)


#get the root node that has a path to at least a  target nodes
best_root_nodes = []

for root in root_nodes:
    for target in target_nodes:
        if nx.has_path(G, root, target):
            best_root_nodes.append(root)
         


if best_root_nodes == []:
    raise Exception('No root node that has a path to all target nodes')
#get the subgraph that follows the path from the root node, we can use BFS

#get all the bgs subgraphs and then combine them
sugraphs = []
for root_node in best_root_nodes:
    subgraph = nx.bfs_tree(G, root_node)
    sugraphs.append(subgraph)

print(len(subgraph.nodes))
#visualize the subgraph, the root node is red, the target nodes are green
#the edges from the root node to the target nodes are blue
net = Network(notebook=True)
net.from_nx(subgraph)

for node in net.nodes:
    if node['id'] in best_root_nodes:
        node['color'] = 'red'
    elif node['id'] in target_nodes:
        node['color'] = 'green'
    else:
        node['color'] = 'gray'

# Get paths from each root node to target nodes
for root_node in best_root_nodes:
    for target in target_nodes:
        if nx.has_path(G, root_node, target):
            path = nx.shortest_path(G, root_node, target)
            # Color the edges in the path
            for i in range(len(path) - 1):
                from_node, to_node = path[i], path[i + 1]
                # Find and color the edge
                for edge in net.edges:
                    if edge['from'] == from_node and edge['to'] == to_node:
                        edge['color'] = 'blue'
                        break

net.show_buttons(filter_=['physics'])
net.show('subgraph.html', notebook=True)


 


Exception: No root node that has a path to all target nodes